In [ ]:
# Install CudaQ package
!pip install cudaq

from IPython.display import clear_output
clear_output()

In [ ]:
!pip install --upgrade cudaq

In [1]:
import cudaq

In [2]:
print(cudaq.__version__)

CUDA-Q Version 0.10.0 (https://github.com/NVIDIA/cuda-quantum f24f4f78673e0bc330fec90a88251321432be970)


In [3]:
import numpy as np
import scipy as sp
from numpy import linalg as LA
from copy import deepcopy
from numpy import matmul
import math
import matplotlib.pyplot as plt
import time

In [4]:
def check_hermitian(mat):
    mat = np.asarray(mat)
    assert np.allclose(mat, mat.T.conjugate(), rtol=1e-05, atol=1e-08), \
    "Sorry! The input matrix should be Hermitian."

In [5]:
#Generate random matrix
def generate_hermitian(n):
    B = np.random.randint(-2, 2, size=(n,n))
    while np.linalg.det(B) == 0:
        B = np.random.randint(-2, 2, size=(n,n))
    Bt = B.transpose()
    A  = matmul(B,Bt)
    matrix_A = matmul(B,Bt)
    # A = matrix_A.tolist()
    return(A)

In [6]:
# Modified version of QFT available on CudaQ documentation: swaps added.
@cudaq.kernel
def QFT(qubits: cudaq.qview):
    '''Args:
    qubits (cudaq.qview): specifies the quantum register to which apply the QFT.'''
    qubit_count = len(qubits)
    # For this specific instance, following swaps should be added
    swap(qubits[1], qubits[qubit_count - 2])
    swap(qubits[0], qubits[qubit_count - 1])
    
    # Apply Hadamard gates and controlled rotation gates.
    for i in range(qubit_count):
        h(qubits[i])
        for j in range(i + 1, qubit_count):
            angle = -(2 * np.pi) / (2**(j - i + 1))
            cr1(angle, [qubits[j]], qubits[i])

# Inverse of QFT can be accessed by applying adjoint operator.
@cudaq.kernel
def invQFT(qubits: cudaq.qview):
    '''Args:
    qubits (cudaq.qview): specifies the quantum register to which apply the inverse QFT.'''
    cudaq.adjoint(QFT, qubits)

In [7]:
def PauliDecomposition(matrix,sparse=False,PauliStringInit="",output="Lists"):
	"""
        Source: https://github.com/HANTLUK/PauliDecomposition/blob/master/TensorizedPauliDecomposition.py
        
		Computes the Pauli decomposition of a square matrix.

		Iteratively splits tensor factors off and decomposes those smaller
		matrices. This is done using submatrices of the original matrix.
		The Pauli strings are generated in each step.

		Args:
			matrix: Matrix to be decomposed
					(Preferrably numpy array/scipysparse).
			output: How the output should be generated.
			sparse: Whether matrix is in sparse format.
			PauliStringInit: For recursive computation.

		Returns:
			decomposition/outString: String of 1XYZ with their factors.
	"""

	matDim = matrix.shape[0]
	qBitDim = math.ceil(np.log(matDim)/np.log(2))

	# Pad, if dimension is not a power of 2

	padDim = 2**qBitDim - matDim
	if padDim != 0:
		if sparse:
			indxptr = np.pad(matrix.indptr, ((0, padDim), (0, padDim)))
			matrix = csr_matrix((matrix.data, matrix.indices, indxptr))
		else:
			matrix = np.pad(matrix, ((0,padDim), (0,padDim)))
	decomposition = []

	if output == "Lists":
		Strings = []
		Coeffs = []


	# Output for dimension 1

	if qBitDim == 0:
		if matrix[0,0] != 0.0:
			if output == "Lists":
				Strings.append(PauliStringInit)
				try:
					Coeffs.append(matrix[0,0])
				except:
					Coeffs.append(matrix[0,0].numpy())
			else:
				decomposition = [f"{matrix[0,0]}, {PauliStringInit}. "]

	# Calculates the tensor product coefficients via the sliced submatrices.
	# If one of these components is zero that coefficient is ignored.

	if qBitDim > 0:
		halfDim = int(2**(qBitDim-1))

		coeff1 = 0.5*(matrix[0:halfDim, 0:halfDim]
						+ matrix[halfDim:, halfDim:])
		coeffX = 0.5*(matrix[halfDim:, 0:halfDim]
						+ matrix[0:halfDim, halfDim:])
		coeffY = -1.j*0.5*(matrix[halfDim:, 0:halfDim]
						- matrix[0:halfDim, halfDim:])
		coeffZ = 0.5*(matrix[0:halfDim, 0:halfDim]
						- matrix[halfDim:, halfDim:])

		coefficients = {"I": coeff1, "X": coeffX, "Y": coeffY, "Z": coeffZ}

		matrix = None

		# Recursion for the Submatrices

		for c in coefficients:
			mat = coefficients[c]
			if sparse:
				nonZero = len(mat.nonzero()[0])
			else:
				nonZero = mat.any()
			# If zero, no branching
			if nonZero != 0:
				subDec = PauliDecomposition(mat,sparse,f"{PauliStringInit}{c}",output)
				if output == "Lists":
					Strings.extend(subDec[0])
					Coeffs.extend(subDec[1])
				else:
					decomposition.append(subDec)

	if output == "Lists":
		return [Strings,Coeffs]
	else:
		outputString = "".join(decomposition)
		return outputString

In [8]:
@cudaq.kernel
def A_exp_pauli(qubits: cudaq.qview, coefficients: list[float], words: list[cudaq.pauli_word], Time: float):
    for i in range(len(coefficients)):
        exp_pauli(coefficients[i]*Time, qubits, words[i])

In [9]:
@cudaq.kernel

def GQLSS(b: list[float], eigs: list[float], r: int, t0: float, coefficients: list[float], words: list[cudaq.pauli_word]):
    """
    A : The matrix representing the linear system.
    b : The vector representing the right-hand side of the linear system.
    t0: A time parameter used in the controlled-Hamiltonian operations.
    r : A parameter used to determine the rotation angles for the ancilla qubit.
    """
    #==========================================================================
    # Preprocessing
    #==========================================================================
    # check_hermitian(A)

    # # Normalize A and b
    # norm_b = LA.norm(b)
    # A = A / norm_b
    # b = b / norm_b

    # # # Calculate condition number and eigenvalues of A
    # kappa = LA.cond(A)
    # eigs = LA.eigvals(A)
    #==========================================================================
    # Quantum Circuit
    #==========================================================================
    b_num = 2 # Should be changed based on the system
    c_num = 4
    # Qubits
    qAnc = cudaq.qubit()
    qReg = cudaq.qvector(4)
    bReg = cudaq.qvector(b_num)
    numQubits = len(qReg) + len(bReg) + 1

    # Classical bits
    cAnc = [0]
    bVec = [0 for i in range(b_num)]
    
    # Apply Hadamard on register C
    h(qReg)

    # Apply Hamiltonian 
    for i in range(len(b)):
        Time = t0/(2**(len(b)-i))
        cudaq.control(A_exp_pauli, qReg[i], bReg, coefficients, words, Time)
    
    # Apply inverse QFT
    QFT(qReg)

    # Swap the qubit
    swap(qReg[1], qReg[c_num - 1])

    # Apply y rotations on Ancilla qubit
    for i in range(len(qReg)+1):
        if i != 0:
            ry.ctrl((2*np.pi)/eigs[i-1], [qReg[i-1]], qAnc)  
        
    # # ================ Uncompute the circuit ================
    mz(qAnc)
    
    # Swap the qubit
    swap(qReg[1], qReg[c_num - 1])

    # Apply QFT
    invQFT(qReg)
    
    # Apply Hamiltonian 
    for i in range(len(b) - 1, -1, -1):
        Time = t0/(2**(len(b)-i))
        cudaq.control(A_exp_pauli, qReg[i], bReg, coefficients, words, (-1)*Time)

    # Apply Hadamard on register C
    h(qReg)

    # Measurement
    mz(bReg)

In [10]:
r_choice = 5

In [11]:
#Define the system
b = np.array([0, 0, -.5, 0])
A = generate_hermitian(4)
kappa = LA.cond(A)
eigs = LA.eigvals(A)
[words,coefficients] = PauliDecomposition(A)

In [12]:
csol = LA.solve(A,b)
csol

array([-2.   , -0.125, -4.5  ,  3.375])

In [13]:
coefficients = [float(np.real(x)) for x in coefficients]

In [14]:
print(words)

['II', 'IX', 'IZ', 'XX', 'XZ', 'YY', 'ZI', 'ZX', 'ZZ']


In [15]:
print(coefficients)

[5.5, 3.5, -0.5, 0.5, -2.0, -0.5, 0.5, -0.5, 0.5]


In [16]:
# Parameters
c_num = 4
b_num = int(np.log2(len(b)))

In [17]:
# Draw the quantum circuit
print("============================== Circuit structure ============================== \n")
print(cudaq.draw(GQLSS, b, eigs, r_choice, 2*np.pi, coefficients, words))

============================== Circuit structure ============================== 



error: 'quake.extract_ref' op invalid constant index value


RuntimeError: cudaq::builder failed to JIT compile the Quake representation.